# Backpropagation

Backpropagation is the most fundamental algorithm on which all other concepts within deep learning relies. At first backpropagation may seem very complicated. However in its' essence, all it is, is a generalization of  can be the [chain-rule](https://en.wikipedia.org/wiki/Chain_rule).

Recall that a neural network (parameterized by weights $W$) predicts a target given an input x i.e. $\hat{y} = p(y|x,W)$. Training a neural network then corresponds to changing the weights $W$ such that the error $E$ between the predicted targets $\hat{y}$ and the true targets $y$ is minimized. 

Neural Networks are usually trained using [gradient descent](https://en.wikipedia.org/wiki/Gradient_descent). Here we update $W_{ij}$ by going in the negative direction of $\frac{\partial E}{\partial w_{ji}}$ with $\alpha$ determining the step size.
$$W_{ij} = W_{ij} - \alpha\frac{\partial E}{\partial w_{ji}}$$

Backpropagation is simply an effective way to calculate the exact gradient of $E$ w.r.t to all the weights in the network.

In practice backpropagation is done with linear algebra. However, showcasing the math behind backpropagation with abstract linear algebra hides the inherent simplicity of the algorithm. Because of this, we will in this notebook only deal with networks which has a single unit in each layer. If you want to dig deeper into how backpropagation handles matrixes of various sizes there are numerous sources online.

Futhermore we will use [Mean Squared error (MSE)](https://en.wikipedia.org/wiki/Mean_squared_error) as our cost function and sigmoid functions as our [activation functions](https://en.wikipedia.org/wiki/Activation_function).

# Forward pass / propagation

In order to do backpropagation we first need to propagate the input through the network from its' input units to its' output units. This is called a forward pass or forward propagation. We need to do this both in order to calculate the error $E$ between our predictions $\hat{y}$ and the actual targets $y$ and to have access the activations. 

If we break forward propagation into its' individual steps it is actually very simple. In each layer we take the input, multiply it by the weights and uses the result of this multiplication as the input to an acitvation function (Here the sigmoid function shown in the lower left corner of the animation). 


![forward propagation](images/forward_propagation.gif)


The animation might be a bit to fast at first, but if you view it a couple of times it should start to make sense. Otherwise we have included all the steps in a static form here:

1) $ s_j = w_{in} \cdot x_{i} $. 

2) $ z_j = \sigma(w_{in} \cdot x_i)$.

3) $ s_k = w_{j} \cdot z_j  $.

4) $ z_k = \sigma(w_j \cdot \sigma(w_{in} \cdot x_i))$.

5) $ s_o = w_{k} \cdot z_k  $.

6) $ z_o = s_o $.

7) $\hat{y}_i = w_k \cdot \sigma(w_j \cdot \sigma(w_{in} \cdot x_{i}))$.

Notice that the functions actually becomes longer and longer, as we get deeper into the network. This is because the later layers are strictly dependent on the former layers, this means that in layer $L_i$ we would in theory need to recalculate all operations from $L_0$ to $L_{i - 1}$. In modern networks this would require emense computational power, which is why we store the result of each operation in memory. This is important to remember, because it is the direct reason neural network has such a big memory requirement compared to other algorithms. 

In [4]:
import numpy as np

"""
Try to construct the forward propagation of the network above
"""

def sigmoid(x):                                        
    return 1 / (1 + np.exp(-x))

x_i = np.array([1]) # The input to the network (Don't touch)
w_in = np.array([2]) # weight at first layer of the network (Don't touch)

s_j = None          # correct this line
z_j = None          # correct this line
w_j = np.array([5]) # weight at second layer of the network (Don't touch)

s_k = None          # correct this line
z_k = None          # correct this line
w_k = np.array([3]) # weight at final layer of the network (Don't touch)

s_o = None          # correct this line
z_o = None          # correct this line

y_hat = None        # correct this line

y_i = np.array([3]) # The target we want the network to predict (Don't touch)
error = [0]        # correct this line

print('The network is correct!' if '{0:0.8f}'.format(error[0]) == '0.00065675' else "Try again, something isn't right")

Try again, something isn't right


After the forward pass, we have access to both the actual targets, and the predictions of the network. Because of this we can calculate the networks' error $E$, which we will do with MSE.
$$E = \frac{1}{2}(\hat{y} - y)^2$$

As noted before, we use backpropagation to figure out to which extend each weight contribute to the total error of the network. Therefore the error is the starting point of the algorithm.

Where forward propagation goes from the beginning of the network to its' end. Backpropagation goes from the end of the network and propagates through it until it reaches the input units (Hence its' name). At each layer it calculates the derivates of the weights using the chain rule combined with partial derivates. If you are not fully comfortable with the chain rule and/or partial derivatives, a great and gentle introduction can be found [here](https://www.youtube.com/watch?v=AXqhWeUEtQU&list=PLSQl0a2vh4HC5feHa6Rc5c0wbRTx56nF7&index=15).

The amount of calculatations needed is quite big for even a toy network like ours. Because of this we have included an animation which goes through all calculations step by step as well as the actual calculations if the animation does not do it for you.
![backpropagation](images/back_propagation.gif)
![backpropagation](images/back_prop.png)

In [13]:
"""
Try to calculate the partial derivatives of the weights defined in last exercise. 
In order to do that you will need the variables used before since the partial derivatives relies on them
"""

def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

partial_W_k  = [0]   # correct this line
partial_W_j  = [0]   # correct this line
partial_W_in = [0]   # correct this line


print('partial_W_k is correct!' if '{0:0.4f}'.format(partial_W_k[0]) == '-0.0358' else "partial_W_k is not correct")
print('partial_W_j is correct!' if '{0:0.4f}'.format(partial_W_j[0]) == '-0.0011' else "partial_W_j is not correct")
print('partial_W_in is correct!' if '{0:0.6f}'.format(partial_W_in[0]) == '0.000021' else "partial_W_in isn't correct")

partial_W_k is not correct
partial_W_j is not correct
partial_W_in isn't correct


As we mentioned initially this might all seem quite complex. However, if you stare at the equations above long enough, you might begin to see a lot of redundancy.

First of all it is easy to see that just like a layer is strictly dependent on all layers before it in forward propagation. It is strictly dependent on all layers after it in backpropagation. This means that doing back propagation in big networks is practically infeasable if we don't store the intemediate results dynamically while we propagate through the network (Here shown as the cache e.g. $C_1$ & $C_2$). 

More importantly we see that the final equation for $\frac{\partial E}{\partial w_{j}} $ and $\frac{\partial E}{\partial w_{in}} $ resembles each other quite a lot. Informally we can rewrite both equations as: 
$$\frac{\partial E}{\partial w_{i}} = \prod_{j=i+1}^L \frac{\partial E}{\partial w_{j}} \cdot w_{i + 1} \cdot d\sigma_{i} \cdot \sigma_{i-1} $$

This equation states that for each layer $L_i$ we multiply the partial derivatives of all layers from the final layer to our current successor $L_{i + 1}$ together. We then multiply by the weights of the successor layer, the derivative of our current activation function and the result of the previous layers' activation function (or the input if we are looking at the first layer in the network).

Althoug the above equation is more general. It still only works informally, and it is almost as confusing as the equations we came from. However, it sheds light on the fact that if we store the partial derivatives of each layer as we iterate through the network we can treat each layer completely seperate from the rest.

# Error signals

The insight that we can treat each layer seperately is the foundation of all modern deep learning frameworks. It is nicely captured if we introduce a new symbol we call the error signal $\delta$ (*delta*). The error signal is simply the accumulated error at each unit. 

Further more we can define a recursive error signal: 

$$ \delta_j = \frac{\partial E}{\partial s_j} $$

In simply put, it is a measure of how much the network error varies with the input to unit $j$. Using the error signal has some nice properties. The most important one being that we can now write back propagation in a simpler and more compact form. 

Let's rewrite all the layers of our network using the error signal:
\begin{align}
\delta_k & = (\hat{y}_i - y_i) \cdot z_k \\
\delta_j & = \delta_k \cdot w_k \cdot \sigma(s_k)(1 - \sigma(s_k)) \cdot z_j \\
\delta_i & = \delta_j \cdot w_j \cdot \sigma(s_j)(1 - \sigma(s_j)) \cdot x_i
\end{align}

It is not far from the original notation, but it lets us define the equation for each layer independent of the proceeding layers in a correct and simple manner. 

# Handling multiple inputs/outputs

The final key to understanding backpropagation is to know how to handle multiple inputs/outpus. We would never have a network with just one unit in each layer. This means that all units in every layer will have multiple inputs and multiple output. 

Luckily for us, it is not very complicated to handle this functionality mathematically. To showcase this we define a new toy network which splits from one units to two and then back to one unit again.

![split network](images/split_network.png)

In [26]:
"""
Try to construct the forward propagation of the splitted network above
Remember that all inputs to a unit is simply summed in forward propagation
"""

x_i = np.array([1])  # The input to the network (Don't touch)
w_in = np.array([2]) # weight at first layer of the network (Don't touch)

s_i = None    # correct this line
z_i = None    # correct this line

w_i_j = np.array([4]) # weight at second layer of the network (Don't touch)
w_i_k = np.array([6]) # weight at second layer of the network (Don't touch)

s_j = None    # correct this line
z_j = None    # correct this line
w_j = np.array([5]) # weight at second layer of the network (Don't touch)

s_k = None    # correct this line
z_k = None    # correct this line
w_k = np.array([3]) # weight at final layer of the network (Don't touch)

s_o = None    # correct this line
z_o = None    # correct this line

y_hat = None  # correct this line

y_i = np.array([5]) # The target we want the network to predict (Don't touch)

error = [0]   # correct this line


print('The network is correct!' if '{0:0.4f}'.format(error[0]) == '1.1371' else "Try again, something isn't right")

Try again, something isn't right


## Multiple inputs

Looking at the network above, it is clear we need to figure out how to split the the error signal into two when going from $o$ to $k$ and $j$. Finding the error signal for $o$ itself is rather simple, since it is the last unit in the network. 

$$ \delta_o = (\hat{y}_i - y_i) $$

Luckily for us, it is also simple to split the error signal, since the two units is added together. Addition in neural networks can be seen as error signal distributors since it takes the error signal and distributes it to all the units in an even manner. This means that the error signal will be distributed to the two units in the following manner.

$$ \delta_j = \delta_o \cdot w_j \cdot \sigma(s_j)(1 - \sigma(s_j))$$
$$ \delta_k = \delta_o \cdot w_k \cdot \sigma(s_k)(1 - \sigma(s_k))$$

Notice that $\delta_o$ is included as it is in both units. This is important to remember why several arcitectures you will hear about later works so well (LSTMs, ResNets etc.). What these architectures have in common is that they try to integrate addition into the networks in order to distribute the error signal / gradient more freely. This tends to have a great impact on  the performance of the networks.


## Multiple outputs

Finally we need to look at what happens in the opposite situation. When a unit have multiple outputs, we simply sum the error signal from each output, just as we sum all inputs of a unit in forward propagation. The error signal for unit $i$ is thus:

$$ \delta_i = \left(\sum_{l \in outs(i)} \delta_l \cdot w_l \right) \cdot \sigma(s_i)(1 - \sigma(s_i)) $$

The last unit in the network is the same as the former network, but defined in terms of the error signal it is:

$$ \delta_{in} = \delta_i \cdot w_{in} \cdot \sigma(x_i)(1 - \sigma(x_i))$$

In [25]:
"""
Try to calculate the partial derivatives of the weights in this new splitted network. 
"""

partial_W_j = [0]     # correct this line

partial_W_k = [0]     # correct this line

partial_W_i_j = [0]   # correct this line

partial_W_i_k = [0]   # correct this line
      
partial_W_in = [0]    # correct this line


print('partial_W_j is correct!' if '{0:0.4f}'.format(partial_W_j[0]) == '0.2099' else "partial_W_j isn't correct")
print('partial_W_k is correct!' if '{0:0.4f}'.format(partial_W_k[0]) == '0.0227' else "partial_W_k isn't correct")
print('partial_W_i_j is correct!' if '{0:0.4f}'.format(partial_W_i_j[0]) == '0.0882' else "partial_W_i_j isn't correct")
print('partial_W_i_k is correct!' if '{0:0.4f}'.format(partial_W_i_k[0]) == '0.0143' else "partial_W_i_k isn't correct")
print('partial_W_in is correct!' if '{0:0.4f}'.format(partial_W_in[0]) == '0.0403' else "partial_W_in isn't correct")

partial_W_j is not correct
partial_W_k is not correct
partial_W_i_j is not correct
partial_W_i_k is not correct
partial_W_in isn't correct
